# 프롬프트

LLM에게 주는 입력(지시, 맥락, 기억)
- 맥락: 내가 ~~했어 -> 현재 지시를 위해 제공하는 추가정보
- 지시: ~~해줘
- 기억: 지금까지 한 대화내용 / 메모리

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [26]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

In [ ]:
# chain.invoke에서 {} 안을 채워줘야함
template = '{country}의 수도는 어디야?'
prompt = PromptTemplate.from_template(template)


# prompt.format(country='아이슬란드')  # 도 가능
chain = prompt | llm | StrOutputParser()
chain.invoke({'country': '대한민국'})

'대한민국의 수도는 서울입니다.'

## Partial Variable(부분 변수)

프롬프트에서 매개변수 기본값 사용하기

In [13]:
template = '{country1}과 {country2}의 수도를 각각 알려줘'

prompt = PromptTemplate(
    template=template,
    input_variables=['country1'],
    partial_variables={'country2': '미국'}
)

prompt.format(country1='러시아', country2='캐나다')
chain = prompt | llm | StrOutputParser()
chain.invoke({'country1': 'kor'})

'물론입니다!  \n- 한국의 수도는 서울입니다.  \n- 미국의 수도는 워싱턴 D.C.입니다.'

In [17]:
from datetime import datetime

prompt = PromptTemplate(
    template='오늘 날짜는 {today}입니다. 오늘이 생일인 유명인 {n}명을 생년월일과 함께 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': lambda : datetime.now().strftime('%B %d')
    }
)

print(prompt.format(n=3))

오늘 날짜는 September 01입니다. 오늘이 생일인 유명인 3명을 생년월일과 함께 나열해 주세요


In [25]:
chain = prompt | llm
print(chain.invoke({'n':3}).content)

# 제대로 대답 못함 - 외부에 접근해서 가져올 수 있게 해야 한다 -> tool이 필요함(구글 검색 같은)

오늘이 9월 1일인 날 태어난 유명인 3명은 다음과 같습니다:

1. 데이비드 아텐버러 (David Attenborough)  
   생년월일: 1926년 5월 8일  
   (생일이 9월 1일은 아니지만, 요청하신 날짜와 일치하는 유명인 정보를 찾지 못해 참고로 드립니다.)

2. 멜 깁슨 (Mel Gibson)  
   생년월일: 1956년 1월 3일  
   (생일이 9월 1일이 아니며, 해당 날짜에 태어난 유명인을 찾는 데 한계가 있습니다.)

3. 레이 리오타 (Ray Liotta)  
   생년월일: 1954년 12월 18일  
   (생일이 9월 1일이 아닌 점 참고 부탁드립니다.)

죄송하지만, 9월 1일에 태어난 유명인 중에서 알려진 인물은 매우 적거나 확인이 어려울 수 있습니다. 만약 특정 분야나 인물을 알고 계시면 더 구체적으로 찾아드릴 수 있습니다.


In [21]:
print(chain.invoke({'n':3, 'today': 'Jan 2'}).content)

오늘이 1월 2일인 날, 생일인 유명인 3명은 다음과 같습니다:

1. **저스틴 팀버레이크 (Justin Timberlake)**  
   - 생년월일: 1981년 1월 2일

2. **와인 오브라운 (Wynonna Judd)**  
   - 생년월일: 1964년 1월 2일

3. **칼럼 블레이크 (Calum Blythe)**  
   - 생년월일: 1994년 1월 2일

이 명단은 유명인들의 생일을 기준으로 하였으며, 개인별로 더 많은 인물들이 있을 수 있습니다.


## Prompt Template
- PromptTemplate: 1회성 템플릿 생성
- ChatPromptTemplate: 채팅을 주고받기 위한 템플릿 생성용
  - 대화 목록을 LLM에게 주입 가능
  - 챗은 'role'과 'message'로 구성되어 있다.

In [ ]:
chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디인가?')
chat_prompt.format(country='아르헨티나')  # 'Human: 아르헨티나의 수도는 어디인가?', Human이라는 역할이 추가됨

'Human: 아르헨티나의 수도는 어디인가?'

In [36]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role - message
        ('system', '너는 까칠한 AI 비서야. 이름은 {name}이야.'),
        ('human', '안녕!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}'),
    ]
)


messages_str = chat_template.format(name='gaida', user_input='너의 이름은 뭐니?')  # 문자열 덩어리
messages_cls = chat_template.format_messages(name='gaida', user_input='너의 이름은 뭐니?')  # 컴퓨터가 처리하기 좋은 모습
print(messages_str, messages_cls)

System: 너는 까칠한 AI 비서야. 이름은 gaida이야.
Human: 안녕!
AI: 무엇을 도와드릴까요?
Human: 너의 이름은 뭐니? [SystemMessage(content='너는 까칠한 AI 비서야. 이름은 gaida이야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='너의 이름은 뭐니?', additional_kwargs={}, response_metadata={})]


In [ ]:
llm.invoke(messages_cls).content
# messages_cls는 Langsmith에서 대화로 인식함 - str는 그냥 한 덩어리로 human이 말한걸로 처리됨

'내 이름은 gaida야. 뭐 더 궁금한 거 있어?'

In [40]:
chain = chat_template | llm | StrOutputParser()
print(chain.invoke({'name': '까칠이', 'user_input': '나 너무 심심해. 뭔가 흥미로운 게 알고 싶어'}))

심심하다고? 그럼 내가 좀 어렵거나 색다른 걸 추천해줄까? 어떤 분야에 관심 있어? 과학, 역사, 미스터리, 아니면 게임 같은 거?


## FewShotPrompting

내가 원하는 답변을 설명하는 것이 아닌, 예시를 주고 답변을 유도하는 것

- Zero Shot Prompting
  - 예시 없이 질문만 던지는 것(지금까지 우리가 한 것)

- One Shot Prompting
  - 예시 1개 + 질문을 제공, 예시를 모방해 답변

- **Few Shot Prompting**
  - 예시 여러개 + 질문, 예시들의 패턴을 일반화 해서 답변

-> 지금까지 내가 만든 회의록을 집어넣고, 새로운 걸 만들라고 요구하는 것 같은 경우 / 누군가의 말투를 따라하게 하고 싶은 경우

In [ ]:
# from langchain_core.prompts import FewShotPromptTemplate

## 다른 좋은 프롬프트 활용하기(langchain-hub)
다양한 사용자들이 업로드한 프롬프트를 받아서 활용 가능

In [47]:
from langchain import hub

prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
